In [ ]:
import numpy as np 
import pandas as pd 
import tensorflow as tf 
import tensorflow.keras.backend as K 
import random
import os
from tensorflow.keras.callbacks import ReduceLROnPlateau 
from sklearn.model_selection import KFold

In [ ]:
tf.device('/gpu:0')

In [ ]:
def seed_everything(seed=42): 
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed) 
    tf.random.set_seed(seed)
    
seed_everything(42)

In [ ]:
train_features = pd.read_csv("../input/lish-moa/train_features.csv")
train_targets = pd.read_csv("../input/lish-moa/train_targets_scored.csv")
test_features = pd.read_csv("../input/lish-moa/test_features.csv")
sample_submission = pd.read_csv("../input/lish-moa/sample_submission.csv")

In [ ]:
train_features=pd.get_dummies(train_features,columns=['cp_type','cp_dose'])
test_features=pd.get_dummies(test_features,columns=['cp_type','cp_dose'])

In [ ]:
del train_features['sig_id']
del test_features['sig_id']
del train_targets['sig_id']

In [ ]:
def build_model():
  model = tf.keras.Sequential() 
  model.add(tf.keras.layers.Input(shape=(train_features.shape[1],))) 
  model.add(tf.keras.layers.BatchNormalization()) 
  model.add(tf.keras.layers.Dense(units=1024,activation='relu')) 

  model.add(tf.keras.layers.BatchNormalization()) 
  model.add(tf.keras.layers.Dropout(0.5)) 
    
  model.add(tf.keras.layers.Dense(units=2048,activation='relu')) 
  model.add(tf.keras.layers.BatchNormalization()) 
  model.add(tf.keras.layers.Dropout(0.5)) 
  model.add(tf.keras.layers.Dense(206, activation="sigmoid"))  

  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy","binary_crossentropy"])
  return model

In [ ]:
pred = np.zeros((train_features.shape[0], 206)) 
pe = np.zeros((test_features.shape[0], 206)) 
train_features = train_features.values
train_targets = train_targets.values

In [ ]:
n_split = 5 
kfoldnumber= 0 

for train_index, validation_index in KFold(n_split).split(train_features):
    kfoldnumber += 1 
    print('Fold number: ',kfoldnumber) 
    
    reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, min_delta=1e-4, mode='min')
    
    net = build_model()
    net.fit(train_features[train_index], train_targets[train_index], batch_size=128, epochs=35, 
            validation_data=(train_features[validation_index], train_targets[validation_index]), verbose=0, callbacks=[reduce_lr_loss])
    
    print("train", net.evaluate(train_features[train_index], train_targets[train_index], verbose=0, batch_size=128))#訓練數據與訓練標籤
    print("val", net.evaluate(train_features[validation_index], train_targets[validation_index], verbose=0, batch_size=128))#驗證數據與驗證標籤
    
    print("predict val...")
    pred[validation_index] = net.predict(train_features[validation_index], batch_size=128, verbose=0)
    print("predict test...")
    pe += net.predict(test_features, batch_size=128, verbose=0) / n_split 

In [ ]:
pe.shape
columns = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
del columns['sig_id']
sub = pd.DataFrame(data=pe, columns=columns.columns)

In [ ]:
sample = pd.read_csv('../input/lish-moa/sample_submission.csv')
sub.insert(0, column = 'sig_id', value=sample['sig_id']) 
sub.to_csv('submission.csv', index=False)